# Tobacco Consumption

Tobacco consumption is one of the primary causes of lung cancer in the World. Tobacco in the form of cigars and cigarettes is usually available to adult population in many supermarkets and grocery stores. The data obtained for this analysis describes Tobacco Consumption in USA from 2000 to 2020. From behavior of the data in those 21 years, the aim of the project is to predict total tobacco consumption in 2021 and 2022. 

At first, the libraries used for this project are imported.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import random
import math
from statsmodels.tsa.seasonal import seasonal_decompose

An additional import is included in order to ignore some warnings while processing the data.

In [ ]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

## Extraction

The data for this project is stored in a *.csv* file. The path to the file is defined in the variable *DATA_PATH*.

In [ ]:
DATA_PATH = "../data/Tobacco_Consumption.csv"

The file is read and a sample of the data is shown.

In [ ]:
tobacco_data_raw = pd.read_csv(DATA_PATH)
tobacco_data_raw.sample(10)

## Exploratory Data Analysis

Describe data table

In [ ]:
tobacco_data_raw.info()

In this table, there are categorial and numerical variables.

The exploration will initially focus on categorical variables and later on the numerical ones. 

### Categorical Data Exploration

The categorical data columns are filtered from the original dataframe.

In [ ]:
# Filter categorical variables from data
tobacco_categorical_data = tobacco_data_raw.select_dtypes(exclude=['int', 'float'])
# Show head of tables
tobacco_categorical_data.head(10)

Categorical data columns are identified.

In [ ]:
# Show numbers of columns
print(f"There is a total  of {len(tobacco_categorical_data.columns)} categorical data columns")
# Show name of the columns
print(f"The columns are: {tobacco_categorical_data.columns}")

To explore the frecuency of elements for each column, frecuency is ploted in a bar chart, where x axis is the name of the elements in the column, and yaxis is the number of times the element is in the column.

In [ ]:
# Create plot object
fig, ax = plt.subplots(2,3, figsize=(20, 15))
fig.subplots_adjust(hspace=.5)
i = 0
# Add subplot of frecuency of elements per column of categociall data
for col in tobacco_categorical_data.columns:
    sns.countplot(tobacco_categorical_data[col], ax=ax[i%2, math.floor(i/2)])
    i+=1
# Rotate axis of each subplot
for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=45)

For *LocationDesc* and *LocationAbbrev* columns there is only one unique value each. Therefore, these columns are constants.

Most values in submeasure have a 21 apperances in the table.

The combinations of values in the columns "Measure", "Submeasure" and "Units" is further explored, to identify how many time each different combinations is shown in the table.

#### Categorical data combinations

Unique combinations of categories are obtained.

In [ ]:
# Get unique combinations by dropping duplicated categorical columns
tobacco_categorical_data.drop_duplicates()

Describe combinations and unique combinations.

In [ ]:
# Get number of unique combinations and total combinations in the table
total_categories_combinations = len(tobacco_categorical_data)
unique_categories_combinations = len(tobacco_categorical_data.drop_duplicates())
# Print summary
print(f"Total combinations of categories (rows): {total_categories_combinations}")
print(f"Find {unique_categories_combinations} unique category combinations")
print(f"Relation: {total_categories_combinations/unique_categories_combinations}")

13 combinations are repeated 21 times in the table.

This number match the number of years in the data. The dataset included 13 different values per year.

### Numerical Data Exploration

The numerical data columns are filtered from the original dataframe.

In [ ]:
# Filter numerical variables from data
tobacco_numerical_data = tobacco_data_raw.select_dtypes(include=['int', 'float'])
# Show head of tables
tobacco_numerical_data.head(10)

Numerical data columns are identified.

In [ ]:
# Show numbers of columns
print(f"There is a total  of {len(tobacco_numerical_data.columns)} numerical data columns")
# Show name of the columns
print(f"The columns are: {tobacco_numerical_data.columns}")

To understand how each variable is related to each other, correlations are obtained and plotted.

In [ ]:
# Explore correlations
correlations = tobacco_numerical_data.corr()
# Plot correlations
sns.heatmap(correlations, annot=True)
plt.show()

*Year* and *Population* have a strong correation with each other, but a low correation to tobacco values.

*Per capita values* have a strong correlation with normal values. 

A test is applied to verify if per capita values are obtained from total values and population.

In [ ]:
# Obtain difference between per capital columns and normal columns divided by population
relation_per_capita = round(tobacco_numerical_data["Total"]/tobacco_numerical_data["Population"], 1) - tobacco_numerical_data["Total Per Capita"]
round(relation_per_capita.median(), 3)

The difference is close to 0. Therefore, the next expressions can be established from the data:

$$
Domestic\_per\_capita= \frac{Domestic}{Population}
$$
$$
Imports\_per\_capita= \frac{Imports}{Population}
$$
$$
Total\_per\_capita= \frac{Total}{Population}
$$

For further analysis, per capita columns are excluded.

*Domestic* and *Imports* have a strong correlation to *Total* column.

In [ ]:
# Difference between total and imports + domestic is obtained
difference_total = tobacco_numerical_data["Total"]- tobacco_numerical_data["Domestic"] - tobacco_numerical_data["Imports"]
difference_total.median()

The difference is 0, so
$$
Total = Imports + Domestic
$$

To have a better understading  of this variables, it is needed to combine numerical exploration with the unique categories exploration. After that analysis, the relation between submeasures is expected to be identified.

### Integrated Exploration (Categories & Numerical Data)

As each year has the same category combinations, one year (2000) is used as a sample. As this analysis focuses in tobacco consumption, only units related to products are taken in to account. Therefore, unit "Pounds" is excluded.

In [ ]:
# Get filtered df 
products_df = tobacco_data_raw[(tobacco_data_raw["Data Value Unit"] != "Pounds") & (tobacco_data_raw["Year"] == 2000)]
products_df

Total Loose Tobacco is compared to Pipe Tobaco and Roll-Your-Own Tobacco 

In [ ]:
# Compare diff between Total Loose Tobacco and Pipe Tobacco
products_df["Domestic"][3] - products_df["Domestic"][7] - products_df["Domestic"][10]

Therefore,
$$
Total\_Loose\_Tobacco = Pipe\_Tobacco + Roll\_Your\_Own\_Tobacco
$$

The Loose Tobacco values are in the table twice (as pounds and as cigarette equivalents), that's the reason the frecuency was the double than other cases in categorical data analysis.

Total Cigars are compared to Small and Large Cigars...

In [ ]:
products_df["Domestic"][2] - products_df["Domestic"][5] - products_df["Domestic"][9]

For Cigars:

$$
Total\_Cigars = Small\_Cigars + Large\_Cigars
$$

In [ ]:
# Get sum of non-total submeasures
sum_cigarettes = products_df["Domestic"][~products_df["Submeasure"].str.contains("Total")].sum()
# Compare sum to Total Combustible Tobacco variable
products_df["Domestic"][products_df["Submeasure"]=="Total Combustible Tobacco"] - sum_cigarettes

$$
Total\_Combustible\_Tobacco = Total\_Cigars + Total\_Loose\_Tobacco o + Cigarette\_Removals
$$

Cigarette, Cigarette Equivalents, and Cigars units have a 1:1:1 relationship.

**Total Combustible Tobacco** contains information of all types of tobacco products submeasures. This value will be the target variable that is going to be predicted in the analysis.

## Data Wrangling

The original dataframe is filtered and transfromed to get a useful table focused in the target variable. Unnecesary columns are drop and year is set as index of the table.

In [ ]:
total_combustible_tobacco_df = tobacco_data_raw[tobacco_data_raw["Submeasure"]=="Total Combustible Tobacco"]
# Drop columns with constant information
total_combustible_tobacco_df.drop(columns=["LocationAbbrev", "LocationDesc", "Topic", "Measure",
    "Submeasure", "Data Value Unit"], inplace=True)
# To reduce data with similar behavior, per capita values will be also ignored in the transformation
total_combustible_tobacco_df.drop(columns=["Domestic Per Capita", "Imports Per Capita", "Total Per Capita"], inplace=True)
# Year to index and datetime object
total_combustible_tobacco_df.set_index("Year", inplace = True)
total_combustible_tobacco_df.index = pd.to_datetime(total_combustible_tobacco_df.index, format = "%Y")
# Show time series
total_combustible_tobacco_df

Plot total over the years

In [ ]:
sns.lineplot(x=total_combustible_tobacco_df.index, y=total_combustible_tobacco_df["Total"])
plt.title("Total Combustible Tobacco per Year")
plt.ylabel("Units")
plt.show()

Store new table as csv.

In [ ]:
# Export ts to df
OUTPUT_PATH = "../data/Transformed_Tobacco_Consumption.csv"
total_combustible_tobacco_df.to_csv(OUTPUT_PATH, index=False)

## Exploration of Transformed Data

In [ ]:
# Explore variables distribution
total_combustible_tobacco_df.describe().convert_dtypes()

Show boxplots and histograms

In [ ]:
# Create boxplots
COLORS = ["b","g", "r", "c", "m", "y"]
fig, ax = plt.subplots(2,2, figsize=(10,7))
i = 0
for col in total_combustible_tobacco_df.columns:
    sns.boxplot(y=col, data=total_combustible_tobacco_df, color = random.choice(COLORS), ax=ax[i%2, math.floor(i/2)])
    i+=1

plt.tight_layout()

For population and total, data seems to be symmetric. However, Domestic and Total are a little right-skewed.

In [ ]:
# Create histogram
COLORS = ["b","g", "r", "c", "m", "y"]
fig, ax = plt.subplots(2,2, figsize=(10,7))
i = 0
for col in total_combustible_tobacco_df.columns:
    sns.distplot(total_combustible_tobacco_df[col], color = random.choice(COLORS), ax=ax[i%2, math.floor(i/2)])
    i+=1

plt.tight_layout()

All variables seem close to be symetric. The previously identified as skewed variables are also close to the center of the data.

Plots all trends by year.

In [ ]:
COLORS = ["b","g", "r", "c", "m", "y"]
fig, ax = plt.subplots(2,2, figsize=(10,7))
i = 0
for col in total_combustible_tobacco_df.columns:
    sns.lineplot(x=total_combustible_tobacco_df.index, y=total_combustible_tobacco_df[col],
        color = random.choice(COLORS), ax=ax[i%2, math.floor(i/2)])
    i+=1
plt.tight_layout()

The percentage of change of variables over time is explored.

### % Change over the years

Get % of change of each variable and plot.

In [ ]:
ts_change_df = total_combustible_tobacco_df.pct_change().dropna()
ts_change_df = round(ts_change_df *100,2)
ts_change_df.head(5)

In [ ]:
# Plot % of change of varibles
COLORS = ["b","g", "r", "c", "m", "y"]
fig, ax = plt.subplots(2,2, figsize=(10,7))
i = 0
for col in ts_change_df.columns:
    sns.lineplot(x=ts_change_df.index, y=ts_change_df[col], color = random.choice(COLORS), ax=ax[i%2, math.floor(i/2)])
    i+=1
plt.tight_layout()

There is no clear behavior related to how much does each vairables changes per year.

### Stationarity

 Augmented Dickey-Fuller test is applied to verify if the data is stationary.

In [ ]:
X = total_combustible_tobacco_df["Total"].values

result = sm.tsa.stattools.adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

The p-value is really small (less than 5% threshold), so it is confirmed the total column is a stationary time series.

### ACF and PACF

In [ ]:
plt.rc("figure", figsize=(10,6))
sm.graphics.tsa.plot_acf(total_combustible_tobacco_df["Total"], lags=20)
plt.show()

In [ ]:
plt.rc("figure", figsize=(10,6))
sm.graphics.tsa.plot_pacf(total_combustible_tobacco_df["Total"], lags=9)
plt.show()

### Time Series Decomposition

In [ ]:
ts_decompose = seasonal_decompose(total_combustible_tobacco_df["Total"], model="additive")
ts_decompose.plot()
plt.show()

There is neither seasonal component or resid.

## Modeling

In this project, three models are compared:
- Linear Regression
- AutoRegresive Integrated Moving Average (ARIMA)
- Holt's Exponential Smoothing

Two metrics are used for comparing and selecting a model:
- Relative Root Mean Square Error (rRMSE)
- Mean Absolute Percentage Error (MAPE)

One of the models will be selected to predict the total tobacco consumption over the next years.

At first, data is splitted in test and train datasets

In [ ]:
train = total_combustible_tobacco_df["Total"][total_combustible_tobacco_df.index < "12-12-2016"]
test = total_combustible_tobacco_df["Total"][total_combustible_tobacco_df.index > "12-12-2016"]

plt.plot(train, color="black")
plt.plot(test, color = "red" )
plt.ylabel("Units")
plt.xlabel("Year")
plt.title("Train/Test split for Tobacco Data")
plt.show()

### Linear Regression Model

### ARIMA Model

### Holt's Exponential Smoothing Model


### Evaluation and Selection

## Results

## Conclusions